In [ ]:
from time import time
import os
import sys
sys.path.append('../')

from utils.codification_cnn import CNNLayer, NNLayer, ChromosomeCNN, FitnessCNN, FitnessCNNParallel
from utils.datamanager import DataManager
from time import time
import numpy as np

In [ ]:
fitness_cnn = FitnessCNN()

# dataset params:
data_folder = '../../../../../../datasets/MNIST_variations'
classes = []

# Fitness params
epochs = 75
batch_size = 128
verbose = 1
redu_plat = False
early_stop = 15
warm_up_epochs= 5
base_lr = 0.001
smooth = 0.1
cosine_dec = True
lr_find = True

dataset = 'MRDBI'
dm = DataManager(dataset, clases=classes, folder_var_mnist=data_folder)
data = dm.load_data()
print(data[0][0].shape)
fitness_cnn.set_params(data, verbose=verbose, batch_size=batch_size, reduce_plateau=redu_plat,
                   epochs=epochs, cosine_decay=cosine_dec, early_stop=early_stop, 
                   warm_epochs=warm_up_epochs, base_lr=base_lr, smooth_label=smooth, find_lr=lr_find)

fitness_folder = '../../delete'
fitness_file = '../../delete/fitness_example' 
fitness_cnn.save(fitness_file)

fitness = FitnessCNNParallel()


In [ ]:
l1 = CNNLayer(99, (7,3), 'leakyreLu', 0.307, 0)
l2 = CNNLayer(192, (5,6), 'relu', 0.271, 0)
l3 = CNNLayer(96, (1,7), 'leakyreLu', 0.041, 1)
l4 = CNNLayer(177, (1,3), 'elu', 0.161, 1)
l5 = CNNLayer(177, (7,4), 'relu', 0.656, 0)
l6 = NNLayer(286, 'elu', 0.432)
l7 = NNLayer(411, 'sigmoid', 0.527)
l8 = NNLayer(621, 'elu', 0.491)
best_mrdbi_v1 = ChromosomeCNN([l1,l2,l3,l4,l5], [l6, l7, l8], fitness)


l1_2 = CNNLayer(86, (3,5), 'leakyreLu', 0.262, 1)
l2_2 = CNNLayer(84, (5,3), 'leakyreLu', 0.319, 1)
l3_2 = CNNLayer(243, (1,3), 'prelu', 0.322, 0)
l4_2 = NNLayer(948, 'sigmoid', 0.467)
l5_2 = NNLayer(780, 'sigmoid', 0.441)
best_mrdbi_v2 = ChromosomeCNN([l1_2, l2_2, l3_2], [l4_2, l5_2], fitness)

In [ ]:
FPS = {16:'FP16 + BN16', 32:'FP32 + BN32', 160:'FP16', 320:'FP32', 3216:'FP32 + BN16'}
cosine_decs = [True, False]

for cos in cosine_decs:
    fitness_cnn.cosine_decay = cos
    fitness_cnn.save(fitness_file)
    for fp in FPS.keys():
        print("\n%s\n" % FPS[fp])
        fitness.set_params(chrom_files_folder=fitness_folder, fitness_file=fitness_file, max_gpus=1,
                   fp=fp, main_line='python /home/daniel/proyectos/Tesis/project/GA/NeuroEvolution/train_gen.py')
        

        ti = time()
        print("Evaluationg First model with FP%d, cosine_decay %s" %(fp, str(cos)) )
        score = fitness.calc(best_mrdbi_v1)
        print("Score: %0.3f" % score)
        print("Elapsed time: %0.3f " % (time() - ti))

        ti = time()
        print("Evaluationg Second model with FP%d, cosine_decay %s" %(fp, str(cos)) )
        score = fitness.calc(best_mrdbi_v2)
        print("Score: %0.3f" % score)
        print("Elapsed time: %0.3f " % (time() - ti))
